In [1]:
import pandas as pd
from astropy.time import Time
import spiceypy as spy
import numpy as np
import rebound as rb
import matplotlib.pyplot as plt
from scipy.optimize import newton
import plotly.express as px
import plotly.graph_objects as go

In [2]:
fireballs = pd.read_csv('datos/cneos_fireball_data_sun_earth.csv', comment = '#')
fireballs

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt),...,earthvz (km/s),sunvx (km/s),sunvy (km/s),sunvz (km/s),x_ecliptic (km),y_ecliptic (km),z_ecliptic (km),vx_ecliptic (km/s),vy_ecliptic (km/s),vz_ecliptic (km/s)
0,2023-08-05 19:54:50,19.9,-131.7,44.0,21.8,3.3,20.4,-6.8,1.719000e+12,3.800,...,-0.002256,0.005726,-0.014173,-0.000012,-3082.442482,5630.849247,-65.885662,18.815052,-11.050763,-2.667611
1,2023-07-26 03:41:54,18.9,-103.4,36.1,15.6,-9.5,12.2,1.9,1.510000e+11,0.440,...,0.000080,0.005511,-0.014239,-0.000006,-1537.707012,-4564.203629,4232.891375,-9.639150,12.258511,-3.219675
2,2023-07-07 16:15:31,14.5,-126.6,38.3,25.7,2.6,4.1,-25.2,2.600000e+10,0.092,...,-0.001285,0.005162,-0.014344,0.000003,4601.122728,4465.436978,-207.684868,-3.095946,-13.558478,-21.580541
3,2023-06-21 19:39:13,12.4,62.8,40.4,16.5,7.3,-2.0,-16.4,5.430000e+11,1.400,...,0.000009,0.004874,-0.014438,0.000009,-328.425133,-5199.387128,3747.474565,-7.971669,-7.562176,-14.576634
4,2023-05-20 11:22:23,-17.8,141.9,29.0,27.9,25.4,6.9,9.3,3.584000e+12,7.200,...,-0.000244,0.004318,-0.014612,0.000023,-6014.375377,-1739.088703,-1352.349152,11.779202,25.659346,-1.017251
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,2004-04-22 04:21:49,-44.3,83.3,29.6,16.3,8.5,-12.1,-6.8,6.240000e+11,1.500,...,-0.001003,0.005269,0.011991,-0.000224,4591.145225,-1829.759390,-4061.952164,-11.167243,-11.910401,-2.242706
289,2004-04-06 03:24:49,-12.0,162.8,35.0,25.6,10.5,-23.2,-2.9,4.330000e+11,1.100,...,0.001208,0.005602,0.011908,-0.000232,4143.918516,3793.835337,-3090.519774,-25.123972,-1.509955,-2.494834
290,2003-11-10 13:54:06,-64.5,136.2,23.0,20.1,14.8,-8.7,10.4,5.180000e+11,1.300,...,-0.001520,0.007926,0.010587,-0.000272,2290.399956,-874.037092,-5894.259476,-11.528905,-7.587885,14.629834
291,2003-09-27 12:59:02,21.0,86.6,26.0,18.2,-1.0,-5.4,-17.3,2.150000e+12,4.600,...,0.000488,0.008589,0.010235,-0.000287,1766.169252,-4336.350206,4365.176443,-1.382563,-2.038180,-17.971748


## bolido de prueba

In [3]:
meteor = fireballs.loc[fireballs['Peak Brightness Date/Time (UT)'].str.contains('2014-01-08 17:05:34')]

In [4]:
meteor

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt),...,earthvz (km/s),sunvx (km/s),sunvy (km/s),sunvz (km/s),x_ecliptic (km),y_ecliptic (km),z_ecliptic (km),vx_ecliptic (km/s),vy_ecliptic (km/s),vz_ecliptic (km/s)
205,2014-01-08 17:05:34,-1.3,147.6,18.7,44.8,-3.4,-43.5,-10.3,3.100000e+10,0.11,...,0.001138,0.010447,0.004187,-0.000242,-5642.441567,2707.874703,-1322.742074,0.08867,-43.752767,7.742526


In [5]:

positions = []

day = 86400
year = 365.25*day
times = np.arange(0,year,day/24/2) #hora a hora
sim = rb.Simulation()
sim.units = 'km', 's', 'kg'
sim.integrator = "WHFast"
sim.dt = 3600
id_met = 205

sun = sim.add(x=meteor['sunx (km)'][id_met], y=meteor['suny (km)'][id_met], z=meteor['sunz (km)'][id_met],
                vx=meteor['sunvx (km/s)'][id_met], vy=meteor['sunvy (km/s)'][id_met], vz=meteor['sunvz (km/s)'][id_met], m=1.98847e30)
earth = sim.add(x=meteor['earthx (km)'][id_met], y=meteor['earthy (km)'][id_met], z=meteor['earthz (km)'][id_met],
                vx=meteor['earthvx (km/s)'][id_met], vy=meteor['earthvy (km/s)'][id_met], vz=meteor['earthvz (km/s)'][id_met], m=5.9722e24)


r_earth = np.array(sim.particles[1].xyz)
v_earth = np.array(sim.particles[1].vxyz)

r_eclip = np.array([meteor['x_ecliptic (km)'][id_met], meteor['y_ecliptic (km)'][id_met], meteor['z_ecliptic (km)'][id_met]])
v_eclip = np.array([meteor['vx_ecliptic (km/s)'][id_met], meteor['vy_ecliptic (km/s)'][id_met], meteor['vz_ecliptic (km/s)'][id_met]])

r_asteroid = r_eclip  + r_earth
v_asteroid = v_eclip + v_earth #así si es

asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2],
                vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

a = 1.496e8
Rsoi = a * (sim.particles[1].m/sim.particles[0].m)**(2/5)

for i in range(len(times)):


    time = -times[i]

    sim.integrate(time)

    r = np.linalg.norm(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))
    v = np.linalg.norm(np.array(sim.particles[2].vxyz) - np.array(sim.particles[1].vxyz))

    print(f'tiempo: {time/3600} r: {r/1e6}')

    if r > Rsoi:
        break


tiempo: -0.0 r: 0.0063968256847276046
tiempo: -0.5 r: 0.08273841584170806
tiempo: -1.0 r: 0.1618926529209932
tiempo: -1.5 r: 0.24105217397731124
tiempo: -2.0 r: 0.32020425911500294
tiempo: -2.5 r: 0.39934996405018525
tiempo: -3.0 r: 0.47849081091289747
tiempo: -3.5 r: 0.5576279547279057
tiempo: -4.0 r: 0.6367622274539667
tiempo: -4.5 r: 0.7158942354086801
tiempo: -5.0 r: 0.7950244311294133
tiempo: -5.5 r: 0.874153160947136
tiempo: -6.0 r: 0.9532806962187688


In [6]:
#Distancia esfera de influencia
print('Rsoi: ', Rsoi)

Rsoi:  924650.722871753


EL bólido atraviesa la esfera de influencia entre t = -5.5 y t = 6.0

In [7]:
Rsoi/v/3600 #tiempo que tardaría en cruzar la esfera de influencia si viajara en linea recta

5.842720829773116

In [8]:
#funcion para encontrar la raiz: punto donde r-Rsoi se hace cero, es decir cuando el bolido cruza la esfera de influencia
def SoiCross(time):

    sim.integrate(-time)
    r = np.linalg.norm(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))

    return r - Rsoi


In [9]:
#Aplicamos newthon raphson para encontrar la raíz
t_soi = newton(SoiCross, 3600, full_output=True)
t_soi


(20948.722431592247,
       converged: True
            flag: 'converged'
  function_calls: 5
      iterations: 4
            root: 20948.722431592247)

In [10]:
t_soi = newton(SoiCross, 3600)
t_soi/3600

5.818994740482141

Aplicando newton raphson nos da que la raíz está en un t=5.861

In [11]:
sim.integrate(-t_soi)
position = np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz)
r = np.linalg.norm(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))
print('Distancia en la que el bolido cruza la esfera de influencia: ', r)

Distancia en la que el bolido cruza la esfera de influencia:  924650.7228717196


In [12]:
def ms(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)


(x_surface_rsoi, y_surface_rsoi, z_suraface_rsoi) = ms(0, 0, 0, Rsoi)
(x_surface_r, y_surface_r, z_suraface_r) = ms(position[0], position[1], position[2], 3e4)
data = [go.Surface(x=x_surface_rsoi, y=y_surface_rsoi, z=z_suraface_rsoi, opacity=0.2, colorscale ='Blues'),
        go.Surface(x=x_surface_r, y=y_surface_r, z=z_suraface_r, opacity=0.8)]
fig = go.Figure(data=data)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()



### For all bodies: find integration time where the bolid cross the influence sphere

In [13]:
times = []

def SoiCross(time):

    sim.integrate(-time)
    r = np.linalg.norm(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))

    return r - Rsoi


for i in range(len(fireballs)):
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "WHFast"
    sim.dt = 3600

    sun = sim.add(x=fireballs['sunx (km)'][i], y=fireballs['suny (km)'][i], z=fireballs['sunz (km)'][i],
                vx=fireballs['sunvx (km/s)'][i], vy=fireballs['sunvy (km/s)'][i], vz=fireballs['sunvz (km/s)'][i], m=1.98847e30)
    earth = sim.add(x=fireballs['earthx (km)'][i], y=fireballs['earthy (km)'][i], z=fireballs['earthz (km)'][i],
                vx=fireballs['earthvx (km/s)'][i], vy=fireballs['earthvy (km/s)'][i], vz=fireballs['earthvz (km/s)'][i], m=5.9722e24)


    r_earth = np.array(sim.particles[1].xyz)
    v_earth = np.array(sim.particles[1].vxyz)

    r_eclip = np.array([fireballs['x_ecliptic (km)'][i], fireballs['y_ecliptic (km)'][i], fireballs['z_ecliptic (km)'][i]])
    v_eclip = np.array([fireballs['vx_ecliptic (km/s)'][i], fireballs['vy_ecliptic (km/s)'][i], fireballs['vz_ecliptic (km/s)'][i]])

    r_asteroid = r_eclip  + r_earth
    v_asteroid = v_eclip + v_earth #así si es

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2],
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

    a = 1.496e8
    Rsoi = a * (sim.particles[1].m/sim.particles[0].m)**(2/5)

    t_soi = newton(SoiCross, 3600)

    times.append(t_soi)


In [14]:
times

[43429.082009397884,
 64471.86752566611,
 36857.73508896958,
 54889.39801304562,
 33202.02610093525,
 62214.8862751712,
 56889.44817778679,
 43084.30321904292,
 19760.12120519465,
 37084.235115154035,
 41706.613281521706,
 65488.1890437781,
 75690.8754695066,
 57430.29884275146,
 60126.9335609471,
 60961.02999609253,
 54587.465110598234,
 44612.78369560152,
 59453.11312476761,
 43777.75920290045,
 30922.411774953667,
 50050.26390558868,
 68163.86172416412,
 58827.3311507706,
 64805.79102745401,
 47287.78822844275,
 48937.4430832754,
 38285.45734383715,
 88574.1349463014,
 47477.272672146726,
 48382.13772085522,
 50520.463254959875,
 54518.43877436453,
 57388.36923122139,
 44094.88324117142,
 84547.31168582095,
 41334.92218091416,
 48868.379188066385,
 41676.20231077769,
 43354.55164726154,
 53733.194177978075,
 49354.89972620929,
 41503.65892450453,
 65182.48722307958,
 33974.00215489509,
 45619.664937757276,
 79469.56642545044,
 71422.15703696711,
 64651.08547253832,
 35636.5561386469

### Bolid position in time integration

In [15]:
positions = []


for i in range(len(fireballs)):
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "WHFast"
    sim.dt = 3600

    sun = sim.add(x=fireballs['sunx (km)'][i], y=fireballs['suny (km)'][i], z=fireballs['sunz (km)'][i],
                vx=fireballs['sunvx (km/s)'][i], vy=fireballs['sunvy (km/s)'][i], vz=fireballs['sunvz (km/s)'][i], m=1.98847e30)
    earth = sim.add(x=fireballs['earthx (km)'][i], y=fireballs['earthy (km)'][i], z=fireballs['earthz (km)'][i],
                vx=fireballs['earthvx (km/s)'][i], vy=fireballs['earthvy (km/s)'][i], vz=fireballs['earthvz (km/s)'][i], m=5.9722e24)


    r_earth = np.array(sim.particles[1].xyz)
    v_earth = np.array(sim.particles[1].vxyz)

    r_eclip = np.array([fireballs['x_ecliptic (km)'][i], fireballs['y_ecliptic (km)'][i], fireballs['z_ecliptic (km)'][i]])
    v_eclip = np.array([fireballs['vx_ecliptic (km/s)'][i], fireballs['vy_ecliptic (km/s)'][i], fireballs['vz_ecliptic (km/s)'][i]])

    r_asteroid = r_eclip  + r_earth
    v_asteroid = v_eclip + v_earth #así si es

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2],
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

    sim.integrate(-times[i])
    positions.append(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))

positions = np.array(positions)

In [16]:
positions

array([[-790904.47512095,  465737.20346631,  111974.01698398],
       [ 566054.24201828, -708447.55545919,  180733.00930105],
       [ 113887.2326628 ,  490010.30454761,  775821.32694184],
       [ 404420.10623097,  388565.57882166,  735146.70929336],
       [-388621.72248639, -838399.83153376,   32217.42051452],
       [ 351794.72069526,  817740.43730979, -250041.34180462],
       [ 327840.89430046,  484369.61382206, -716160.56862559],
       [ -76498.33793044,  -26600.28411938,  921096.99097718],
       [ 552078.10811105, -724610.63606758,  158518.69050165],
       [ 827098.35156786, -406662.33522318,   74250.89210703],
       [ 806303.79560272, -432615.16720894,  133031.93246224],
       [-398960.01421109,  788311.23753376, -272719.80725614],
       [ 611891.13846694,  328708.21030888,  610344.31405757],
       [ -94777.88941097,  914885.46049607,  -94769.62609401],
       [  76398.57017913, -467776.8803426 , -793931.74996933],
       [ 270831.9109238 , -341001.89695516,  815688.287

In [17]:
df = pd.DataFrame(columns=['x', 'y', 'z', 'r'])

df.x = positions[:,0]
df.y = positions[:,1]
df.z = positions[:,2]
r = [3e4 for j in range(len(positions[:,2]))]
df.r = r

In [18]:
fig = px.scatter_3d(df, x='x', y='y', z='z', opacity=0.3)

# tight layout
fig.update_traces(marker_size = 5)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [19]:

(x_earth_surface, y_earth_surface, z_earth_surface) = ms(0, 0, 0, Rsoi)
# Create a scatter plot
x_scatter = df.x  # Example scatter data
y_scatter = df.y
z_scatter = df.z

# Create 3D plot
fig = go.Figure()

# Solid sphere
fig.add_trace(go.Surface(x=x_earth_surface, y=y_earth_surface, z=z_earth_surface, colorscale='blues', opacity=0.5))

# Scatter plot
fig.add_trace(go.Scatter3d(x=x_scatter, y=y_scatter, z=z_scatter, mode='markers', marker=dict(size=3, color='red')))

fig.update_layout(scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=0))
fig.show()

## EN UNIDADES CANONICAS

In [20]:
G = sim.G
UL = 1.496e+8
UM = sim.particles[0].m
UT = (UL**3/(G*UM))**(1/2)

In [22]:
positions_canonic = positions/UL

In [24]:
df = pd.DataFrame(columns=['x', 'y', 'z', 'r'])

df.x = positions_canonic[:,0]
df.y = positions_canonic[:,1]
df.z = positions_canonic[:,2]
r = [3e4/UL for j in range(len(positions_canonic[:,2]))]
df.r = r

In [25]:

(x_earth_surface, y_earth_surface, z_earth_surface) = ms(0, 0, 0, Rsoi/UL)

x_scatter = df.x  # Example scatter data
y_scatter = df.y
z_scatter = df.z

# Create 3D plot
fig = go.Figure()

# Solid sphere
fig.add_trace(go.Surface(x=x_earth_surface, y=y_earth_surface, z=z_earth_surface, colorscale='blues', opacity=0.5))

# Scatter plot
fig.add_trace(go.Scatter3d(x=x_scatter, y=y_scatter, z=z_scatter, mode='markers', marker=dict(size=3, color='red')))

fig.update_layout(scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=0))
fig.show()